In [ ]:
## LOAD MODULES ##
import json
import librosa
import numpy as np
import os
from skimage.feature import peak_local_max
import torch
from tqdm import tqdm
import utils
import model.net as net

In [ ]:
model_dir = 'experiments/trained_model'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mel_frequencies = librosa.mel_frequencies(n_mels=128, fmin=400.0, fmax=6000.0, htk=True)
json_path = os.path.join(model_dir, 'params.json')
params = utils.Params(json_path)
model = net.Net(params).to(device)

restore_path = os.path.join(model_dir, 'pretrained.pth.tar')
_ = utils.load_checkpoint(restore_path, model, optimizer=None)

files = librosa.util.find_files('data/newfiles')
detections = {}
for file in tqdm(files):
    basename = os.path.basename(file)
    detections[basename] = []
    
    x, sr = librosa.load(file, sr=24000, mono=True)
    frames = librosa.util.frame(x, frame_length=4*sr, hop_length=4*sr, axis=0) # split audio file into 4s clips
    for i in range(frames.shape[0]):
        audio_clip = torch.tensor(frames[i]).view(1,-1).to(device)
        batch = {"input": audio_clip}
        with torch.no_grad():
            output_dict = model(batch)
        
        predictions = output_dict['logits'][0,0,:,:].sigmoid().cpu().numpy()
        peak_idx = peak_local_max(predictions, min_distance=21, threshold_abs=0.5) # defualt detection threshold of 0.5
        for peak in range(peak_idx.shape[0]):
            mels, frame = peak_idx[peak]
            score = predictions[mels,frame]
            freq_hz = mel_frequencies[mels]
            time = (i*4) + librosa.frames_to_time(frame, sr=sr, hop_length=256)
            detections[basename].append({'Time [s]': f"{time:.2f}", 'Score': f"{score:.2f}", 'Frequency [Hz]': f"{freq_hz:.2f}"})
    if not detections[basename]:
        detections.popitem()

# write detections to a json file
with open("batch_detections.json", "w") as outfile:
    json.dump(detections, outfile)